# --------------------------------------------------------------
# Day 20 NFL.py 🏈: Webscraping for college data
# --------------------------------------------------------------

# Necessary data cleaning

In [ ]:
import nfl_data_py as nfl

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [18]:
import nfl_data_py as nfl

import pandas as pd
import numpy as np

years = range(2010, 2023)
data = nfl.import_weekly_data(years)
draft = nfl.import_draft_picks(years)
df = data[(data['season'] >= 2010) & (data['position'] == 'QB')]

rounds = [draft[draft['pfr_player_name'] == plyr]['round'] for plyr in df['player_display_name']]

df_draft = draft[(draft['round'] == 1) & (draft['position'] == 'QB')][['pfr_player_name', 'season', 'round', 'pick', 'team']]
df_draft.loc[df_draft['team'] == 'STL', 'team'] = 'LA'

labels = [df[df['player_display_name'] == plyr] for plyr in df_draft['pfr_player_name']]

players_by_season = df.groupby(['player_display_name', 'season']).agg({'recent_team':lambda x: x.iloc[0]}).reset_index()

labels = {}
for plyr in df_draft['pfr_player_name']:
    draft_team = np.squeeze(df_draft[df_draft['pfr_player_name'] == plyr]['team'])
    draft_year = np.squeeze(df_draft[df_draft['pfr_player_name'] ==plyr]['season'].values[0])
    labels[plyr] = players_by_season[(players_by_season['player_display_name'] == plyr) & (players_by_season['recent_team'] == draft_team)].shape[0] >= 5

Downcasting floats.


# Data scraping

In [56]:
test_df = pd.DataFrame({
    'player':list(labels.keys()),
    'successful':list(labels.values())
})

In [57]:
test_df['player'].values

array(['Sam Bradford', 'Tim Tebow', 'Cam Newton', 'Jake Locker',
       'Blaine Gabbert', 'Christian Ponder', 'Andrew Luck',
       'Robert Griffin III', 'Ryan Tannehill', 'Brandon Weeden',
       'EJ Manuel', 'Blake Bortles', 'Johnny Manziel',
       'Teddy Bridgewater', 'Jameis Winston', 'Marcus Mariota',
       'Jared Goff', 'Carson Wentz', 'Paxton Lynch', 'Kenny Pickett',
       'Trevor Lawrence', 'Zach Wilson', 'Trey Lance', 'Justin Fields',
       'Mac Jones', 'Joe Burrow', 'Tua Tagovailoa', 'Justin Herbert',
       'Jordan Love', 'Kyler Murray', 'Daniel Jones', 'Dwayne Haskins',
       'Baker Mayfield', 'Sam Darnold', 'Josh Allen', 'Josh Rosen',
       'Lamar Jackson', 'Mitchell Trubisky', 'Patrick Mahomes',
       'Deshaun Watson'], dtype=object)

In [58]:
test_df.loc[test_df['player'] == 'Ryan Tannehill', 'successful'] = True
test_df.loc[test_df['player'] == 'Jared Goff', 'successful'] = True
test_df.loc[test_df['player'] == 'Trevor Lawrence', 'successful'] = True
test_df.loc[test_df['player'] == 'Joe Burrow', 'successful'] = True
test_df.loc[test_df['player'] == 'Tua Tagovailoa', 'successful'] = True
test_df.loc[test_df['player'] == 'Justin Herbert', 'successful'] = True
test_df.loc[test_df['player'] == 'Jordan Love', 'successful'] = True
test_df.loc[test_df['player'] == 'Kyler Murray', 'successful'] = True
test_df.loc[test_df['player'] == 'Baker Mayfield', 'successful'] = True
test_df.loc[test_df['player'] == 'Josh Allen', 'successful'] = True
test_df.loc[test_df['player'] == 'Lamar Jackson', 'successful'] = True
test_df.loc[test_df['player'] == 'Patrick Mahomes', 'successful'] = True
test_df.loc[test_df['player'] == 'Deshaun Watson', 'successful'] = True

In [48]:
print(test_df[test_df['successful'] == True].shape)
print(test_df[test_df['successful'] == False].shape)

(18, 2)
(22, 2)


In [62]:
df_draft['player'] = df_draft['pfr_player_name']
df_draft = df_draft.drop('pfr_player_name', axis=1)
df = pd.merge(test_df, df_draft,on='player', how='inner')

In [ ]:
Player()

In [91]:
from sportsipy.ncaaf.roster import Player

blough = Player('david-blough-1')  # Currently pulling career stats
print(blough.passing_yards)  # Prints Blough's CAREER passing yards total
# Prints Blough's passing yards total only for the 2017 season
print(blough('2017').passing_yards)
# Prints Blough's passing touchdowns for the 2017 season only
print(blough.passing_touchdowns)

None
None
None


In [80]:
AbstractPlayer('jared-goff-1', 'Jared Goff', 'https://www.sports-reference.com/cfb/sdf/jared-goff-1.html').adjusted_yards_per_attempt()

AttributeError: 'AbstractPlayer' object has no attribute '_index'

In [68]:
players_by_season

,player_display_name,season,recent_team
0,A.J. Feeley,2011,LA
1,AJ McCarron,2015,CIN
2,AJ McCarron,2017,CIN
3,AJ McCarron,2018,LV
4,AJ McCarron,2019,HOU
...,...,...,...
974,Will Grier,2019,CAR
975,Zach Mettenberger,2014,TEN
976,Zach Mettenberger,2015,TEN
977,Zach Wilson,2021,NYJ


In [238]:
import requests
from bs4 import BeautifulSoup

In [291]:
def extract_passing_table(player_name):

    first_name = player_name.split(' ')[0].lower()
    last_name = player_name.split(' ')[1].lower()
    player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-1.html'
    html_content = requests.get(player_url).text
    
    if len(player_name.split(' ')) > 2:
        first_name = player_name.split(' ')[0].lower()
        last_name = player_name.split(' ')[1].lower()
        suffix = player_name.split(' ')[2].lower()
        player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-{suffix}-1.html'
        html_content = requests.get(player_url).text
    
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', {'id': 'passing_standard'})
    
    if not table:
        print("Passing stats table not found")
        return None

    baseline_headers = []
    headers = [th.getText() for th in table.find_all('th')]
    rows = []
    
    for tr in table.find_all('tr')[1:]:
        cells = [td.getText() for td in tr.find_all('td')]
        if cells: 
            rows.append(cells)

    column_names = [
    'G',        
    'Cmp',      
    'Att',      
    'Cmp%',     
    'Yds',      
    'TD',       
    'TD%',      
    'Int',      
    'Int%',     
    'Y/A',      
    'AY/A',     
    'Y/C',      
    'Y/G',      
    'Rate',     
    'name'
    ]
    
    totals = rows[len(rows)-1]
    totals = [stat for stat in totals if stat.strip()]
    totals.append(player_name)

    if len(totals) == len(column_names):
        final = pd.DataFrame({name: [value] for name, value in zip(column_names, totals)})
    else:
        print("Error: Number of stats does not match number of column names.")

    return final

In [358]:
x = extract_passing_table('Cam Newton')

Passing stats table not found


In [253]:
lens = []
for name in df['player']:
    lens.append(len(name.split(' ')))

In [255]:
pd.value_counts(lens)

/var/folders/k_/y5wzmsx1483f9z9t9tjtdj1m0000gn/T/ipykernel_61123/3900589874.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(lens)
/var/folders/k_/y5wzmsx1483f9z9t9tjtdj1m0000gn/T/ipykernel_61123/3900589874.py:1: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.value_counts(lens)


2    39
3     1
Name: count, dtype: int64

In [248]:
'Robert Griffin III'.split(' ')[2].lower()

'iii'

In [250]:
x = extract_passing_table('Robert Griffin III')

In [308]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_html_content(player_url):
    response = requests.get(player_url)
    if response.status_code == 200:
        return response.text
    else:
        print("Failed to retrieve the page")
        return None

In [231]:
x